In [1]:
import numpy
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, CrossEncoder

In [3]:
text = open("input.txt").read()

In [4]:
length = 600
overlap = 200

start = 0
snippets = []

while start < len(text):
    end = start + length
    snippets.append(text[start:end])
    start += length - overlap

In [2]:
embedding_model = SentenceTransformer("intfloat/e5-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
embeddings = []

for snippet in tqdm(snippets):
    embedding = embedding_model.encode("passage: " + snippet)
    embeddings.append(embedding)

100%|██████████| 1507/1507 [00:27<00:00, 55.32it/s]


In [6]:
numpy.save("embeddings.npy", embeddings)

In [ ]:
embeddings = numpy.load("embeddings.npy")

In [13]:
query = "How much money did the narrator pay to Fullerton after returning to New York?"

In [14]:
query_embedding = embedding_model.encode("query: " + query)
similarities = cosine_similarity([query_embedding], embeddings)
top_100_indices = numpy.argsort(similarities[0])[-100:][::-1]
top_100_snippets = [snippets[i] for i in top_100_indices]

In [9]:
reranker_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [15]:
reranker_inputs = []

for snippet in top_100_snippets:
    reranker_inputs.append((query, snippet))

In [16]:
scores = []

for input in tqdm(reranker_inputs):
    score = reranker_model.predict(input)
    scores.append(score)

100%|██████████| 100/100 [00:00<00:00, 171.33it/s]


In [17]:
top_5_reranked_indices = numpy.argsort(scores)[-5:][::-1]
top_5_snippets = [snippets[top_100_indices[i]] for i in top_5_reranked_indices]

for i, snippet in enumerate(top_5_snippets):
    print(f"Snippet {i+1}: {snippet}\n")
    print("-----------------")

Snippet 1:  take your business. Understand?”

There was no sense in getting mad or trying to argue, so I went back
to the hotel, paid my bill and took the first train back to New York.
It was tough. I wanted to take back some real money and that Teller
wouldn’t let me make even one trade.

I got back to New York, paid Fullerton his five hundred, and started
trading again with the St. Louis money. I had good and bad spells, but
I was doing better than breaking even. After all, I didn’t have much to
unlearn; only to grasp the one fact that there was more to the game of
stock speculation than I had consider

-----------------
Snippet 2: until I was cleaned out. I went to old Fullerton and got him to stake
me to five hundred dollars. And I came back from St. Louis, as I told
you, with money I took out of the bucket shops there--a game I could
always beat.

I played more carefully and did better for a while. As soon as I was in
easy circumstances I began to live pretty well. I made friends 